# Analyzing images
This notebook will build on what we did in the training notebook. First, let us consider the following experiment: We have a group of TRAP2 mice, and we are interested in studying whether the basolateral amygdala (BLA) is involved in two different behavioral tasks. To do so, we induced the expression of tdTomato fluorescent protein in TRAP2 mice by injecting 4OH-tamoxifen during the first behavioral task, labeling the cells activated during this period. After a week of expression, we performed the second different behavioral task, perfused the mice, and performed immunological labeling of cFos protein expression. We took photos of the BLA with a fluorescent microscope, segmented the BLA region, and saved the TIFF images in the folders `tdt` and `cfos`.


We will start by importing the necessary packages for our analysis, including the functions from `cellrake`. Ensure you have correctly installed `cellrake` in your Conda environment following the instructions in [README.md](../README.md) and that you are running this notebook with this environment (top right corner of the notebook if you are using VSCode).

In [1]:
from pathlib import Path
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

from cellrake.main import analyze
from cellrake.predicting import colocalize

## Analyzing images with a trained model
In this first example, we will use a pre-trained model to predict tdTomato-positive cells and want to use it to identify positive cells in a new set of images. We will first load the model and establish the path where we store the images to analyze.

In [ ]:
model_path = Path('./sample_data/tdt_model_rf')
with open(model_path, "rb") as file:
    model = pkl.load(file)

tdt_folder = Path("./sample_data/tdt")

The `analyze` function takes three arguments: `image_folder`, `model`, `threshold_rel`, and `cmap`. To obtain more information, read the docstring of this function typing `help(analyze)`. We will run it using the defined path to the images and the uploaded machine-learning model. The selected `cmap` is "Reds" as the tdTomato protein glows in red under the microscope.

In [ ]:
analyze(tdt_folder, model, threshold_rel=0.1, cmap='Reds')

We have just predicted the tdTomato-positive cells in the sample dataset! In the output folder, named `tdt_analysis`, we have the results of the analysis:

- `labelled_images`: the rendered images comparing the original image with the identified tdTomato-positive cells overlapped.
- `counts`: the number of predicted cells for each image.
- `detected`: the information of the predicted cells for each image.
- `features`: teh features of each predicted cell (e.g., area, perimeter, eccentricity, intensity, etc.)
- `tdt_segmentation`: the segmented images before applying the model.

If you train a new model and you want to apply it to this dataset, you won't have to segment again the images because CellRake will automatically detect the `tdt_segmentation` file.

We can examine the results checking the results table and opening an image:

In [ ]:
pd.read_csv("./sample_data/tdt_analysis/counts.csv")

In [ ]:
pil_image = Image.open("./sample_data/tdt_analysis/labelled_images/C2-34453_B1_10X_BLA_001.png")
plt.imshow(pil_image)
plt.axis('off')
plt.show()

# Analyzing colocalization

We may sometimes work with images in which cells express multiple markers. For instance, in the provided example dataset, cells can be tdTomato-positive and cFos-positive. In this case, analyzing which cells express both markers simultaneously makes sense. To do so, we first have to analyze both markers separately (as we have already done) and call the function `colocalize`.

In [28]:
cfos_rois_processed = Path('./sample_data/cfos_analysis/detected.pkl')
cfos_images = Path('./sample_data/cfos')
tdt_rois_processed = Path('./sample_data/tdt_analysis/detected.pkl')
tdt_images = Path('./sample_data/tdt')

In [ ]:
colocalize(cfos_rois_processed, cfos_images, tdt_rois_processed, tdt_images)

The analysis results are automatically saved in the folder named `colocalization_cfos_tdt`.

In [ ]:
pd.read_csv("./sample_data/colocalization_cfos_tdt/colocalization_results.csv")

In [ ]:
pil_image = Image.open("./sample_data/colocalization_cfos_tdt/labelled_images/34453_B1_10X_BLA_001.png")
plt.imshow(pil_image)
plt.axis('off')
plt.show()